In [1]:
var openDigger = require('../src/open_digger');
const fs = require('fs');

// 时间范围：2024年7月至2025年6月
const startYear = 2024;
const startMonth = 7;
const endYear = 2025;
const endMonth = 6;

// 生成月份标签数组（格式："24-07"）
const months = [];
for (let year = startYear; year <= endYear; year++) {
    const start = (year === startYear) ? startMonth : 1;
    const end = (year === endYear) ? endMonth : 12;
    for (let month = start; month <= end; month++) {
        const yearStr = year.toString().slice(-2);
        const monthStr = month.toString().padStart(2, '0');
        months.push(`${yearStr}-${monthStr}`);
    }
}

var limit = 15; // 每月返回前15名公司

async function saveGlobalMonthlyRankToCSV() {
    const companyData = {};
    const allCompanies = new Set();

    // 循环处理每个月
    for (let year = startYear; year <= endYear; year++) {
        const start = (year === startYear) ? startMonth : 1;
        const end = (year === endYear) ? endMonth : 12;
        
        for (let month = start; month <= end; month++) {
            // 生成当前月份标签
            const yearStr = year.toString().slice(-2);
            const monthStr = month.toString().padStart(2, '0');
            const dateStr = `${yearStr}-${monthStr}`;
            
            console.log(`📅 正在处理 ${dateStr} 全球数据...`);

            // 使用标签筛选而非直接构造WHERE子句，避免IN语句过长
            let data = await openDigger.index.openrank.getRepoOpenrank({
                labelIntersect: ['Company'],  // 仅筛选公司类型，不限制地区
                startYear: year,
                startMonth: month,
                endYear: year,
                endMonth: month,
                groupBy: 'Company',
                groupTimeRange: 'month',
                order: 'DESC',
                limit: limit
            });

            // 输出表格到控制台
            console.table(data.map((d, i) => ({
                '排名': i + 1,
                '公司': d.name,
                '影响力': d.openrank[0]
            })));

            // 保存当月排名
            data.forEach((d, index) => {
                const rank = index + 1;
                allCompanies.add(d.name);
                if (!companyData[d.name]) {
                    companyData[d.name] = {};
                }
                companyData[d.name][dateStr] = rank;
            });
        }
    }

    // 准备CSV内容
    let csvContent = ['公司名称', ...months].join(',') + '\n';

    // 添加各公司数据
    allCompanies.forEach(company => {
        const row = [company];
        months.forEach(month => {
            row.push(companyData[company][month] || '');
        });
        csvContent += row.join(',') + '\n';
    });

    // 保存为CSV文件
    const fileName = `全球企业月度排名数据_${startYear}${startMonth}_${endYear}${endMonth}.csv`;
    fs.writeFileSync(fileName, csvContent, 'utf8');
    console.log(`✅ 全球数据已成功保存到 ${fileName}`);
}

// 执行函数
saveGlobalMonthlyRankToCSV().catch(err => {
    console.error('❌ 处理过程中发生错误:', err);
});

📅 正在处理 24-07 全球数据...


Promise { <pending> }

┌─────────┬──────┬──────────────────┬──────────┐
│ (index) │ 排名 │ 公司             │ 影响力   │
├─────────┼──────┼──────────────────┼──────────┤
│ 0       │ 1    │ 'Microsoft'      │ 15142.74 │
│ 1       │ 2    │ 'Huawei'         │ 9193.53  │
│ 2       │ 3    │ 'Google'         │ 5335.42  │
│ 3       │ 4    │ 'Amazon'         │ 4615.72  │
│ 4       │ 5    │ 'Meta'           │ 2919.24  │
│ 5       │ 6    │ 'RedHat'         │ 2330.93  │
│ 6       │ 7    │ 'Mozilla'        │ 2284.28  │
│ 7       │ 8    │ 'Alibaba'        │ 2106.11  │
│ 8       │ 9    │ 'IBM'            │ 1815.33  │
│ 9       │ 10   │ 'Elastic'        │ 1763.89  │
│ 10      │ 11   │ 'HashiCorp'      │ 1575.43  │
│ 11      │ 12   │ 'Intel'          │ 1570.38  │
│ 12      │ 13   │ 'Ant group'      │ 1537.58  │
│ 13      │ 14   │ 'Grafana Labs'   │ 1527.89  │
│ 14      │ 15   │ 'Nabu Casa Inc.' │ 1493.35  │
└─────────┴──────┴──────────────────┴──────────┘
📅 正在处理 24-08 全球数据...
┌─────────┬──────┬──────────────────┬──────────┐
│ (ind